In [7]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
import quaternion
import re
import datetime
import os
import glob
from MOF_build.functions.rotate import rotate
from MOF_build.functions.read import read
from MOF_build.functions.output import output

In [8]:
'''translate and rotate node to beginning point'''

#node = read.pdb('PCN222_lib_nodeZr.pdb')
node = read.pdb('node_lib.pdb')
#print(node.iloc[30,:],node.iloc[57,:])
array = node.loc[:, ['x','y','z']].to_numpy()
center = 0.5*(array[47]+array[46]) #center point (beginning point)
new_node = array-center
#uio66
#any_tdx, any_tdy, any_tdz = (
#np.array([1, 0, 0]),
#np.array([0, 1, 0]),
#np.array([0, 1, 2]))

#PCN222
any_tdx, any_tdy, any_tdz = (
    np.array([1, 0, 0]),
    np.array([0, 1, 0]),
    np.array([0, 1, 1]))
dx, dy, dz = np.array([1, 0, 0]), np.array([0, 1, 0]), np.array([0, 0, 1])
carte_basis = np.vstack([dx, dy, dz])
  
tric_basis = rotate.coordinate_transfer(any_tdx, any_tdy, any_tdz, carte_basis)
#v1_file = new_node[9]-new_node[0]
#v2_file = new_node[27]-new_node[18]
v1_file = new_node[47]-new_node[46]
v2_file = new_node[40]-new_node[16]
v1_frame = tric_basis[2]+tric_basis[1]
v2_frame = tric_basis[0]
rotated_node = rotate.rotate_twice_linker(node,center,v2_file,v2_frame,v1_file,v1_frame)
#rotateq_x90=rotate.calculate_q_rotation_with_axis_degree(tric_basis[0],np.pi/2)
#rotated_node = rotate.get_rotated_array(rotated_node0,rotateq_x90)
node.loc[:, ['x','y','z']] = rotated_node
newdf = pd.concat([node.iloc[:,0:3],node.iloc[:,6],node.iloc[:,3:6]],axis=1, join = 'outer') #move ['NOTE'] to column3
newdf.to_csv('PCNZRRnodeOO.txt',header=None,sep='\t',index=False)
output('PCNZRRnodeOO',True,False,False)
print(newdf)

   Atom_label Residue  Res_number  Note             x         y         z
0          O2     MOL           2   O1- -1.353000e+00 -2.459659  2.932953
1          O1     MOL           2   O1- -2.460000e+00 -0.530884  3.132195
2          C2     MOL           2     C -3.216000e+00 -2.389790  4.372425
3          C1     MOL           2     C -2.253000e+00 -1.727662  3.408607
4          O2     MOL           2   O1- -1.353000e+00  2.459659 -2.932953
5          O1     MOL           2   O1- -2.460000e+00  0.530884 -3.132195
6          C2     MOL           2     C -3.216000e+00  2.389784 -4.371425
7          C1     MOL           2     C -2.253000e+00  1.727662 -3.408607
8          O2     MOL           2   O1- -1.353000e+00 -3.777982  0.615542
9          O1     MOL           2   O1- -2.460000e+00 -2.963805 -1.143328
10         C2     MOL           2     C -3.216000e+00 -4.978958 -0.180094
11         C1     MOL           2     C -2.253000e+00 -3.812448 -0.256694
12         O2     MOL           2   O1

In [9]:
#reset resnumber  
justnode = read.pdb('PCNZRRnodeOO.pdb') #ZRRnode.pdb
justnode.iloc[:,2]=1 #set resnumber as 1
#print(justnode)
df = pd.concat([justnode.iloc[:,0:3],justnode.iloc[:,6],justnode.iloc[:,3:6]],axis=1, join = 'outer')
df.to_csv('lib_ZRnode.txt',header=None,sep='\t',index=False)
output('lib_ZRnode',True,False,False)


In [10]:
'''add dummy atom to Zr node'''
pre_dummy_pdb = newdf

atom_type = []
for i in pre_dummy_pdb['Atom_label']:
    atom_type.append(re.sub(r"\d", "", i))

pre_dummy_pdb.insert(7, "atom_type", atom_type, True)

df_Zr =pre_dummy_pdb[pre_dummy_pdb['atom_type']=='Zr'].reset_index()
df_O =pre_dummy_pdb[pre_dummy_pdb['atom_type']=='O'].reset_index()
O_positions = df_O.loc[:, ['x','y','z']].to_numpy()

df_add_dummy=pd.DataFrame()
for i in range(df_Zr.shape[0]):
    df_dummy = read.pdb("dummy_template.pdb")
    distance_O = []
    distance_Osort = []
    pointZR = df_Zr.loc[i, ['x','y','z']].to_numpy()
    for j in range(len(O_positions)):
        distance_O.append(np.linalg.norm(pointZR-O_positions[j]) )
        distance_Osort.append(np.linalg.norm(pointZR-O_positions[j]) )
    distance_Osort.sort()
    threshhold = distance_Osort[8]
    indices = [index for index, num in enumerate(distance_O) if num < threshhold]
    #print(indices,distance_O)
    dummy_atoms = np.zeros((8,3))
    row =0
    for k in indices:
        newD = pointZR+(O_positions[k]-pointZR)/np.linalg.norm(pointZR-O_positions[i])
        dummy_atoms[row,:]=newD
        row +=1
    print(pointZR,'\n')
    dummy_atoms=np.round(dummy_atoms,3)
    df_dummy.loc[0, ['x','y','z']]=pointZR
    df_dummy.loc[1:, ['x', 'y', 'z']] = dummy_atoms
    df_dummy.loc[:,'Res_number'] = i+1
  
    df_add_dummy = pd.concat([df_add_dummy,df_dummy],ignore_index=True)

df_add_dummy



[-1.7600000000000016 0.8722905509231133 1.533065375243421] 

[-1.7600000000000018 -0.8722905509231149 -1.533065375243421] 

[1.7599999999999982 0.8722905509231131 1.5330653752434211] 

[1.7599999999999985 -0.8722905509231149 -1.533065375243421] 

[0.0 2.136758089073905 -1.2159671536588594] 

[1.1102230246251565e-16 -2.136758089073907 1.2159671536588628] 



,Atom_label,Residue,Res_number,x,y,z,Note
0,Zr,ZRR,1,-1.760000e+00,0.872291,1.533065,Zr
1,D1,ZRR,1,-1.952000e+00,0.486000,1.973000,D
2,D2,ZRR,1,-1.952000e+00,1.447000,1.426000,D
3,D3,ZRR,1,-1.709000e+00,1.172000,2.060000,D
4,D4,ZRR,1,-2.364000e+00,0.852000,1.498000,D
5,D5,ZRR,1,-1.762000e+00,0.347000,1.274000,D
6,D6,ZRR,1,-1.762000e+00,0.918000,0.949000,D
7,D7,ZRR,1,-1.276000e+00,1.171000,1.374000,D
8,D8,ZRR,1,-1.276000e+00,0.583000,1.709000,D
9,Zr,ZRR,2,-1.760000e+00,-0.872291,-1.533065,Zr


In [11]:
'''output'''
df_add_dummy.insert(3, "note", df_add_dummy['Note'], True)
df_add_dummy.to_csv('0adddummy.txt',header=None,sep='\t',index=False)
#
output('0adddummy',True,False,False)